# User interface Node #

In [1]:
#! /usr/bin/env python3

import rospy
import actionlib
import actionlib.msg
from assignment_2_2022.srv import Ngoal
import assignment_2_2022.msg
import time
import sys
import select
import os   
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout
import logging
from IPython.display import display

In [2]:
def D_cb(b):
    if ((act_clnt.get_state()) == 1):
        act_clnt.cancel_goal() # delete the goal
        n_deleted = rospy.get_param('n_deleted') # get the actual number of goal deleted
        n_deleted = n_deleted + 1 # increase it
        rospy.set_param('n_deleted', n_deleted) # load it in the paramserver

In [3]:
def N_cb(Gx, Gy):
    if (Gx.disabled and Gy.disabled):
        Gx.disabled = False
        Gy.disabled = False     
        N.disabled = True

In [4]:
def NG_cb(b):
    
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    logging.debug("NG_CB")
    
    Gx.disabled = True
    Gy.disabled = True
    N.disabled = False  
    goal.target_pose.pose.position.x = float(Gx.value)
    goal.target_pose.pose.position.y = float(Gy.value)
    act_clnt.send_goal(goal)
    

In [5]:
def I_cb(b):
    info = srv_info()
    Gr.value = info.n_goal
    Gd.value = info.n_deleted

In [6]:
def UI(): 
    # different layout:
    L1 = Layout(width = 'auto', height = '10%', min_width = '80px', min_height = '54px', max_width = '160px')
    L2 = Layout(width = 'auto', height = '5%', min_width = '40px', min_height = '20px', max_width = '80px')
    
    # delete goal button:
    D = widgets.Button(value = False, description = 'Delete Goal', disabled = False, button_style = 'danger', layout = L1, continuous_update = False)
    
    # new goal button:
    N = widgets.Button(value = False, description = 'New Goal', disabled = False, button_style = 'success', layout = L1, continuous_update = False)
    
    # information button:
    I = widgets.Button(value = False, description = 'Information', disabled = False, button_style = 'info', layout = L1, continuous_update = False)
    
    # info box for the goal coordinates:
    # name:
    Gx_n = widgets.Text(value = 'x_goal', disabled = True, layout = L2)
    Gy_n = widgets.Text(value = 'y_goal', disabled = True, layout = L2)
    # data:
    Gx = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    Gy = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    # combination:
    Gx_c = VBox([Gx_n, Gx])
    Gy_c = VBox([Gy_n, Gy])
    Gxy = HBox([Gx_c, Gy_c])
    
    
    # info box for the goal reached and deleted: FloatText
    # name:
    Gr_n = widgets.Text(value = 'goal_r', disabled = True, layout = L2)
    Gd_n = widgets.Text(value = 'goal_d', disabled = True, layout = L2)
    # data:
    Gr = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    Gd = widgets.FloatText(value = '0.00', disabled = True, layout = L2, continuous_update = False)
    # combination:
    Gr_c = VBox([Gr_n, Gr])
    Gd_c = VBox([Gd_n, Gd])
    Grd = HBox([Gr_c, Gd_c])
    
    # constraints on the dimensions: 0.00
    
    # button callbacks:
    D.on_click(D_cb)
    N.on_click(N_cb)
    # when i've typed what needed in the text boxes call the cb NG_cb()
    #Gx.observe(NG_cb)
    #Gy.observe(NG_cb, names = 'value')
    I.on_click(I_cb)
    
    
        
    # combine everything
    UI = VBox([D, HBox([N, Gxy]), HBox([I, Grd])])
    
    
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    logging.debug("display")

    display(UI)

In [ ]:
def main():
    # ROS node
    gf = 0 # goal flag to increase the number of goal only the first time the state pass to succeeded
    goal = assignment_2_2022.msg.PlanningGoal() 
    goal.target_pose.pose.position.z = 0.0
    info = Ngoal()
    n_goal = 0 # number of goal reached
    n_deleted = 0 # number of goal deleted
    
    rospy.init_node('user_interface', anonymous=True) # init the node
    act_clnt = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
    act_clnt.wait_for_server()
    srv_info = rospy.ServiceProxy("/info", Ngoal)
    rospy.wait_for_service("/info")
    
    rospy.set_param('n_goal', n_goal) # setting the parameters in the paramserver
    rospy.set_param('n_deleted', n_deleted) # setting the parameters in the paramserver
    
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    logging.debug("befui")

    UI()
        
    while True:
        
        logger = logging.getLogger()
        logger.setLevel(logging.DEBUG)
        logging.debug("befui")
        
        
        
        logger = logging.getLogger()
        logger.setLevel(logging.DEBUG)
        logging.debug("afui")
        
        if (act_clnt.get_state() == 3 and gf == 0): # if i've reached the goal
            n_goal = rospy.get_param('n_goal') # get the actual number of goal
            n_goal = n_goal + 1 # increase it
            rospy.set_param('n_goal', n_goal) # load it in the paramserver
            gf = 1
        elif (act_clnt.get_state() != 3):
            gf = 0
        else:
            continue
        
main()
    